In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
from bs4 import BeautifulSoup
import pickle
from sklearn.utils import shuffle
import tensorflow
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential , Model
from keras.layers import Dense, Embedding, GRU, LSTM ,SpatialDropout1D, Bidirectional, Dropout , Input 
from scipy.stats import mode
from sklearn.utils import shuffle

In [3]:
def read_xml(name):
  with open(name, 'r') as f:
    data = f.read()
  lines = data.splitlines()
  revie_text =[]
  for i in range(0,len(lines)):
    if(lines[i]=='<review_text>'):
      revie_text.append(lines[i+1])
  return revie_text

In [4]:
def flat_list(list):
  flat_list = []
  for sublist in list:
    for item in sublist:
        flat_list.append(item)
  return flat_list

In [5]:
def preprocess(lines):
  words = lines.split()
  lower=[]
  for i in words:
    lower.append(i.lower())
  return lower

In [6]:
dataset='/content/drive/MyDrive/sorted_data_acl'
folders = ["dvd","kitchen_&_housewares","books", "electronics"]
files = ["negative.review","positive.review"]
os.chdir(dataset)
pos_data = []
neg_data = []
labels = []
f = []

for filePath in os.listdir():
 if filePath not in folders:
   continue
 f.append(filePath)

print(f)
for i in range(0,len(f)):
  for j in range(0,len(files)):
    if(j==0):
       neg_data.append(dataset+"/"+f[i]+"/"+files[j])
    if(j==1):
      pos_data.append(dataset+"/"+f[i]+"/"+files[j])
print(pos_data)
print(neg_data)

['dvd', 'kitchen_&_housewares', 'books', 'electronics']
['/content/drive/MyDrive/sorted_data_acl/dvd/positive.review', '/content/drive/MyDrive/sorted_data_acl/kitchen_&_housewares/positive.review', '/content/drive/MyDrive/sorted_data_acl/books/positive.review', '/content/drive/MyDrive/sorted_data_acl/electronics/positive.review']
['/content/drive/MyDrive/sorted_data_acl/dvd/negative.review', '/content/drive/MyDrive/sorted_data_acl/kitchen_&_housewares/negative.review', '/content/drive/MyDrive/sorted_data_acl/books/negative.review', '/content/drive/MyDrive/sorted_data_acl/electronics/negative.review']


In [7]:
train_pos_data =[]
test_pos_data =[]
train_neg_data =[]
test_neg_data =[]
for i in range(len(folders)):
  if(folders[i]=="kitchen_&_housewares"):
    test_pos_data.append(pos_data[i])
    test_neg_data.append(neg_data[i])
  else:
    train_pos_data.append(pos_data[i])
    train_neg_data.append(neg_data[i])

In [8]:
print(train_pos_data)
print(test_pos_data)
print(train_neg_data)
print(test_neg_data)

['/content/drive/MyDrive/sorted_data_acl/dvd/positive.review', '/content/drive/MyDrive/sorted_data_acl/books/positive.review', '/content/drive/MyDrive/sorted_data_acl/electronics/positive.review']
['/content/drive/MyDrive/sorted_data_acl/kitchen_&_housewares/positive.review']
['/content/drive/MyDrive/sorted_data_acl/dvd/negative.review', '/content/drive/MyDrive/sorted_data_acl/books/negative.review', '/content/drive/MyDrive/sorted_data_acl/electronics/negative.review']
['/content/drive/MyDrive/sorted_data_acl/kitchen_&_housewares/negative.review']


In [9]:
train_data = train_neg_data + train_pos_data
test_data = test_neg_data + test_pos_data

In [10]:
print(train_data)
print(test_data)

['/content/drive/MyDrive/sorted_data_acl/dvd/negative.review', '/content/drive/MyDrive/sorted_data_acl/books/negative.review', '/content/drive/MyDrive/sorted_data_acl/electronics/negative.review', '/content/drive/MyDrive/sorted_data_acl/dvd/positive.review', '/content/drive/MyDrive/sorted_data_acl/books/positive.review', '/content/drive/MyDrive/sorted_data_acl/electronics/positive.review']
['/content/drive/MyDrive/sorted_data_acl/kitchen_&_housewares/negative.review', '/content/drive/MyDrive/sorted_data_acl/kitchen_&_housewares/positive.review']


In [11]:
train_pos_lines = []
train_neg_lines = []
for i in range(0,len(train_pos_data)):
  train_pos_lines.append(read_xml(train_pos_data[i]))
for i in range(0,len(train_neg_data)):
  train_neg_lines.append(read_xml(train_neg_data[i]))
print(train_pos_lines)
print(train_neg_lines)
list = flat_list(train_pos_lines)
print(list[90])
list_2 = flat_list(train_neg_lines)
print(list_2[90])

[["I'm not sure why Sony, which now owns I Dream of Jeannie, decided to colorize the first season of this series.  Whatever the reason, you can readily tell by looking at the prices here on Amazon.com that the original black-and-white version of the first season is worth a lot more.  The reason for that is simple--I Dream of Jeannie was originally broadcast in black-and-white.  And for a television fan like myself, that's the ONLY way to watch the first season.", "If you don't own this dvd you need to add it to your collection. In my opinion it is the best american animated film ever released", "Denzel Washington is great but Angelina Jolie he is even better. The movie itself is pretty scary, to tell you the truth it's not pleasant. There are some parts you don't really need to see. The plot of the movie is interesting and keeps you wondering what is gonna happen next. I recommend watching it with somebody, cause it's not pretty. But it's a good movie.", 'I rediscovered this after a lo

In [12]:
print(len(list))
print(len(list_2))

3000
3000


In [13]:
print(list[2999])
print(list_2[2999])

The GPS works in both my Dell Axim X5 and in my HP Compaq nc6000 with a compact flash adapter card.  I have used it with MS Streets and Trips and ArcPad
I bought this for easy transfer of pictures from my digital camera with SD memory card anywhere not home and sometimes from other peoples memory card (xD and memory stick)..


In [14]:
train_pos_words =[]
for i in range(0,len(list)):
  train_pos_words.append(preprocess(list[i]))

train_neg_words =[]
for i in range(0,len(list_2)):
  train_neg_words.append(preprocess(list_2[i]))

In [15]:
print(train_pos_words[2999])
print(train_neg_words[2999])

['the', 'gps', 'works', 'in', 'both', 'my', 'dell', 'axim', 'x5', 'and', 'in', 'my', 'hp', 'compaq', 'nc6000', 'with', 'a', 'compact', 'flash', 'adapter', 'card.', 'i', 'have', 'used', 'it', 'with', 'ms', 'streets', 'and', 'trips', 'and', 'arcpad']
['i', 'bought', 'this', 'for', 'easy', 'transfer', 'of', 'pictures', 'from', 'my', 'digital', 'camera', 'with', 'sd', 'memory', 'card', 'anywhere', 'not', 'home', 'and', 'sometimes', 'from', 'other', 'peoples', 'memory', 'card', '(xd', 'and', 'memory', 'stick)..']


In [16]:
train_pos_words = flat_list(train_pos_words)
print(len(train_pos_words))
train_neg_words = flat_list(train_neg_words)
print(len(train_neg_words))

221225
230758


In [17]:
y_train_pos = []
y_train_neg = []
for i in range(0,len(train_pos_words)):
  y_train_pos.append(1)
for i in range(0,len(train_neg_words)):
  y_train_neg.append(0)
print(len(y_train_pos))
print(len(y_train_neg))

221225
230758


In [18]:
test_pos_lines = []
test_neg_lines = []
for i in range(0,len(test_pos_data)):
  test_pos_lines.append(read_xml(test_pos_data[i]))
for i in range(0,len(test_neg_data)):
  test_neg_lines.append(read_xml(test_neg_data[i]))
print(test_pos_lines)
print(test_neg_lines)
list_3 = flat_list(test_pos_lines)
print(list_3[90])
list_4 = flat_list(test_neg_lines)
print(list_4[90])

[['For those of you unfamiliar with the "A Series of Unfortunate Events" books, they detail the absurdly tragic lives of the fictional Baudelaire orphans, and their struggles to overcome adversity (after adversity, after adversity, ad infinitum).  Hovering in the background of their tragedies and occasional, brief glimpses of happiness, is their distant cousin, the scheming, greedy, conniving, ruthless cousin, the Count.', 'i absolutely love this product. my neighbor has four little yippers and my shepard/chow mix was antogonized by the yipping on our side of the fence. I hung the device on my side of the fence and the noise keeps the neighbors dog from picking "arguments" with my dog. all barking and fighting has ceased. all the surrounding neighbor as well as me can get a good nights sleep now', 'I purchased 3 units and they work well for us.  I have 2 units in children\'s bedrooms on AC power and the 3rd in the kitchen on batteries for portability.  The sound is clear and set-up and

In [19]:
print(len(list_3))
print(len(list_4))

1000
1000


In [20]:
print(list_3[999])
print(list_4[999])

We have three dogs so the fur is endless. I come home from work and start the Roomba and voila, no more dog hair. It picks up more stuff than my other vacuums ever did.
Worked great the first couple of times.  I didn't keep it on the charger for 24 hours (not realizing that this is a problem) and now the battery only works for 10 minutes at a time.  It's now basically worthless


In [21]:
test_pos_words =[]
for i in range(0,len(list_3)):
  test_pos_words.append(preprocess(list_3[i]))

test_neg_words =[]
for i in range(0,len(list_4)):
  test_neg_words.append(preprocess(list_4[i]))

In [22]:
print(test_pos_words[999])
print(test_neg_words[999])

['we', 'have', 'three', 'dogs', 'so', 'the', 'fur', 'is', 'endless.', 'i', 'come', 'home', 'from', 'work', 'and', 'start', 'the', 'roomba', 'and', 'voila,', 'no', 'more', 'dog', 'hair.', 'it', 'picks', 'up', 'more', 'stuff', 'than', 'my', 'other', 'vacuums', 'ever', 'did.']
['worked', 'great', 'the', 'first', 'couple', 'of', 'times.', 'i', "didn't", 'keep', 'it', 'on', 'the', 'charger', 'for', '24', 'hours', '(not', 'realizing', 'that', 'this', 'is', 'a', 'problem)', 'and', 'now', 'the', 'battery', 'only', 'works', 'for', '10', 'minutes', 'at', 'a', 'time.', "it's", 'now', 'basically', 'worthless']


In [23]:
test_pos_words = flat_list(test_pos_words)
print(len(test_pos_words))
test_neg_words = flat_list(test_neg_words)
print(len(test_neg_words))

62392
68242


In [24]:
y_test_pos = []
y_test_neg = []
for i in range(0,len(test_pos_words)):
  y_test_pos.append(1)
for i in range(0,len(test_neg_words)):
  y_test_neg.append(0)
print(len(y_test_pos))
print(len(y_test_neg))

62392
68242


In [79]:
x_train = train_pos_words + train_neg_words
y_train = y_train_pos + y_train_neg

In [89]:
x_test = test_pos_words + test_neg_words
y_test = y_test_pos + y_test_neg

In [90]:
x_train,y_train = shuffle(x_train, y_train)
x_test,y_test = shuffle(x_test, y_test)

In [91]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [92]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

451983
451983
130634
130634


In [93]:
t  = Tokenizer()
t.fit_on_texts(x_train)
word_index = t.word_index
print("The word index", word_index)
x_train = t.texts_to_sequences(x_train)
t_2  = Tokenizer()
t_2.fit_on_texts(x_test)
x_test = t_2.texts_to_sequences(x_test)
print(len(x_train))
print(len(x_test))

The word index {'the': 1, 'and': 2, 'a': 3, 'i': 4, 'to': 5, 'of': 6, 'is': 7, 'it': 8, 'this': 9, 'in': 10, 'that': 11, 'for': 12, 'was': 13, 'with': 14, 'on': 15, 'you': 16, 'but': 17, 'have': 18, 'not': 19, 'as': 20, 'my': 21, 'book': 22, 'are': 23, 'one': 24, 'be': 25, 'all': 26, 'so': 27, 'from': 28, 'at': 29, 'or': 30, 'if': 31, 'they': 32, 'about': 33, 'an': 34, 'movie': 35, 'just': 36, 'by': 37, 'like': 38, 'has': 39, 'his': 40, 'very': 41, 'he': 42, 'good': 43, 'out': 44, 'when': 45, 'what': 46, 'would': 47, 'more': 48, 'me': 49, 'had': 50, 'there': 51, 'up': 52, 'who': 53, 'great': 54, 'no': 55, 'read': 56, 'can': 57, 'some': 58, "it's": 59, 'get': 60, 'time': 61, 'only': 62, 'will': 63, 'her': 64, 'film': 65, 'first': 66, 'which': 67, 'well': 68, 'do': 69, 'your': 70, 'much': 71, 'other': 72, 'than': 73, "don't": 74, 'even': 75, 'them': 76, 'how': 77, 'after': 78, 'these': 79, 'really': 80, 'been': 81, 'because': 82, 'were': 83, 'we': 84, 'their': 85, 'dvd': 86, 'work': 87, 

In [94]:
x_train = flat_list(x_train)
x_test = flat_list(x_test)

In [161]:
sentences = []
labels = []
i=0
j=150
for word in range(len(y_train)):
  if(j>= len(y_train)):
    break
  sent= x_train[i:j:]
  label = y_train[i:j:]
  sentences.append(sent)
  labels.append(label)
  i=i+150
  j=j+150
print(len(sentences))
print(len(labels)) 

3013
3013


In [164]:
print(sentences[2000])
print(labels[2000])

[4, 510, 413, 193, 44, 2, 21924, 62, 5, 7150, 2, 3, 11, 24, 24, 434, 7, 21925, 1592, 118, 2094, 75, 180, 61, 6, 1, 85, 1141, 4, 4, 30, 1535, 3462, 22, 3, 1177, 99, 1, 21926, 3, 1, 9, 5, 3, 8936, 10259, 74, 10, 5, 19, 20, 2092, 319, 10, 1180, 58, 98, 766, 45, 1730, 92, 2, 9635, 3293, 10, 1331, 6414, 575, 9, 23, 2774, 91, 1, 219, 3, 742, 201, 1, 4, 767, 1269, 12, 12228, 87, 4, 4378, 1, 56, 6, 653, 108, 4134, 2116, 5876, 1947, 134, 9049, 433, 485, 23, 3, 3738, 2728, 266, 59, 2702, 69, 4, 91, 234, 374, 25, 685, 4, 560, 85, 450, 118, 54, 1731, 43, 73, 2, 13, 865, 3, 22, 3996, 2296, 43, 2936, 1155, 2, 563, 1022, 4856, 21927, 1228, 1, 119, 181, 2, 231, 48, 318, 4, 2, 4, 3144, 53]
[0 1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 0
 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0
 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 1 1
 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1
 0 0]


In [165]:
train_sentences = np.array(sentences)
train_labels = np.array(labels)
print(train_sentences.shape)
print(train_labels.shape)

(3013, 150)
(3013, 150)


In [166]:
train_lables_for_each_sentence =[]
for i in range(train_labels.shape[0]):
    most_similar = mode(train_labels[i]) 
    most_similar = most_similar.mode[0]
    train_lables_for_each_sentence.append(most_similar)
train_lables_for_each_sentence=np.array(train_lables_for_each_sentence)
print(train_lables_for_each_sentence.shape)

(3013,)


In [167]:
test_sentences = []
test_labels = []
i=0
j=150
for word in range(len(y_test)):
  if(j>= len(y_test)):
    break
  sent= x_test[i:j:]
  label = y_test[i:j:]
  test_sentences.append(sent)
  test_labels.append(label)
  i=i+150
  j=j+150
print(len(test_sentences))
print(len(test_labels)) 

870
870


In [169]:
print(test_sentences[100])
print(test_labels[100])

[4, 1111, 1, 14, 38, 380, 4, 1, 43, 52, 4930, 111, 1112, 5, 36, 189, 531, 10, 55, 6, 591, 142, 3, 26, 82, 725, 1166, 5, 7, 163, 122, 1099, 598, 6, 232, 81, 8, 4, 23, 1113, 1, 570, 12, 90, 6, 182, 92, 130, 61, 4, 13, 1210, 2885, 1109, 4931, 48, 5, 47, 94, 11, 28, 316, 3, 152, 9, 3, 146, 193, 63, 103, 16, 3, 69, 8, 353, 53, 116, 2, 285, 1, 452, 2, 2, 35, 1212, 1877, 39, 2886, 566, 55, 14, 569, 5, 184, 1497, 3, 11, 9, 988, 35, 19, 4932, 5, 24, 993, 2111, 78, 1878, 29, 12, 5, 579, 2, 2, 191, 209, 154, 4933, 2430, 24, 919, 64, 125, 14, 3, 42, 242, 7, 195, 2, 55, 21, 76, 30, 566, 68, 12, 4934, 51, 1, 1100, 3578, 178, 30, 16, 2, 22, 24, 89, 1]
[1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 1
 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1 1 0 1
 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0
 0 1]


In [170]:
test_sentences = np.asarray(test_sentences)
test_labels = np.array(test_labels)
print(test_sentences.shape)
print(test_labels.shape)

(870, 150)
(870, 150)


In [171]:
test_lables_for_each_sentence =[]
for i in range(test_labels.shape[0]):
    most_similar = mode(test_labels[i]) 
    most_similar = most_similar.mode[0]
    test_lables_for_each_sentence.append(most_similar)
test_lables_for_each_sentence=np.array(test_lables_for_each_sentence)
print(test_lables_for_each_sentence.shape)

(870,)


In [40]:
import gensim
model =  gensim.models.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/model.bin" , binary=True) 

In [41]:
vec=model['human']
print(vec)

[-0.215811 -0.729229 -0.031277  0.783592 -0.311612 -0.020156  0.455803
  0.078662 -0.429745 -0.080442  0.55945   0.132717  0.385656  0.327662
  0.407918  1.002462  0.029467 -0.39079  -0.255424 -0.307728 -0.255069
 -0.055123 -0.034864 -0.940706  0.493973 -0.055644 -0.324368 -0.294988
  0.305286  0.018529  0.263904 -0.077612 -0.089076 -0.012973 -0.692759
 -0.893139  0.099085  0.733702  0.693438  0.64574   0.352023 -0.397965
 -0.490918  0.816374  0.413414  0.528836 -0.426027 -0.38141   0.477267
 -0.372627 -0.80783  -0.399647 -0.318791 -1.550399  0.391485 -0.045143
 -0.019567 -0.353307  0.455406  0.404592 -0.104873  0.547805 -0.039498
  0.760495 -0.585712  0.231003  0.346273 -0.34873  -0.524779 -0.055544
 -0.21562  -0.009868  0.159058  1.365654 -0.073559 -0.57399   0.379082
  0.254269 -1.004791  0.122549 -0.274344 -0.289617  0.311391 -0.48021
  1.159809  0.239188  0.005173  1.397103  0.318408  0.007176 -0.64621
 -0.254797 -0.096981  0.358251 -0.181356  0.456904 -0.012477  0.420092
 -0.1708

In [58]:
w2v_embeddings_index={}
for word in model.wv.vocab:
    w2v_embeddings_index[word] = model[word]
print("Loaded %s word vectors."% len(w2v_embeddings_index))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Loaded 273930 word vectors.


In [59]:
embedding_dim = 300
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = w2v_embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Embedding Matrix shape: ', embedding_matrix.shape)

Embedding Matrix shape:  (26349, 300)


In [172]:
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=150,
                            trainable=False)

In [173]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 150, 300)          7904700   
                                                                 
 conv1d_6 (Conv1D)           (None, 144, 32)           67232     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 144, 32)          0         
 1D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 4608)              0         
                                                                 
 dense_10 (Dense)            (None, 500)               2304500   
                                                                 
 dropout_4 (Dropout)         (None, 500)               0         
                                                      

In [176]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.05 ), metrics=['accuracy'])
model.fit(train_sentences, train_lables_for_each_sentence, batch_size=64, epochs=150 , validation_data=(test_sentences, test_lables_for_each_sentence))

Epoch 1/150


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


48/48 [==============================] - 8s 150ms/step - loss: 2.9778 - accuracy: 0.7468 - val_loss: 15.5763 - val_accuracy: 0.7023
Epoch 2/150
48/48 [==============================] - 7s 148ms/step - loss: 2.0603 - accuracy: 0.7577 - val_loss: 16.4076 - val_accuracy: 0.6851
Epoch 3/150
48/48 [==============================] - 7s 148ms/step - loss: 2.6272 - accuracy: 0.7594 - val_loss: 16.8171 - val_accuracy: 0.6713
Epoch 4/150
48/48 [==============================] - 7s 149ms/step - loss: 1.8723 - accuracy: 0.7640 - val_loss: 18.9418 - val_accuracy: 0.6736
Epoch 5/150
48/48 [==============================] - 7s 148ms/step - loss: 2.3048 - accuracy: 0.7707 - val_loss: 20.5493 - val_accuracy: 0.6885
Epoch 6/150
48/48 [==============================] - 7s 147ms/step - loss: 2.2648 - accuracy: 0.7660 - val_loss: 18.7389 - val_accuracy: 0.6943
Epoch 7/150
48/48 [==============================] - 7s 149ms/step - loss: 1.2787 - accuracy: 0.7766 - val_loss: 21.8007 - val_accuracy: 0.7034
Epoc

In [177]:
loss, accuracy = model.evaluate(test_sentences, test_lables_for_each_sentence)
print("Accuracy: %f" % (accuracy*100))

28/28 [==============================] - 1s 23ms/step - loss: 365.5962 - accuracy: 0.7057
Accuracy: 70.574713


In [180]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(train_sentences, train_lables_for_each_sentence)

BernoulliNB()

In [181]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(test_sentences)

In [183]:
acc = accuracy_score(test_lables_for_each_sentence, y_pred)

In [184]:
print("Accuracy: %f" % (acc*100))

Accuracy: 73.793103
